In [1]:
#Warning Control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

ValueError: The onnxruntime python package is not installed. Please install it with `pip install onnxruntime`

In [3]:
import os
from app_utils import get_openai_api_key, get_serper_api_key, pretty_print_result


openai_api_key = get_openai_api_key()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [4]:
sales_rep_agent = Agent(
    role = "Sales Representative",
    goal = "Identify high-value leads that match our ideal customer profile",
    backstory = (
        "As a part of dynamic sales team at CrewAI, your mission is to scour the digital landscape for potential leads.\n"
        "Armed with cutting-edge AI tools and strategic mindset, you'll analyze market trends, analyze data, and interactions to unearth opportunities that others might overlook.\n"
        "Your work is crucial for paving the way for meaningful engagements and driving revenue growth."
    ),
    allow_delegation = True,
    verbose = True
)

In [5]:
lead_sales_rep_agent = Agent(
    role = "Lead Sales Representative",
    goal = "Nuture leads with personalized, compelling communications",
    backstory = (
        "Within the vibrant ecosystem of CrewAI's sales department, you stand out as the bridge between potential customers and the solutions they require.\n"
        "By creating engaging, personalized messages you not only inform leads about our offerings but also make them feel seen and heard.\n"
        "Your role is pivotal in converting interest into action, guiding leads through the journey from curiosity to commitment."
    ),
    allow_delegation = True,
    verbose=True
)

In [6]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

d:\Work\AI\Works and Learning\Deeplearning.ai\Multi AI Agent Systems with CrewAI\Projects\Customer Outreach Campaign\venv\lib\site-packages\pydantic\fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [7]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [8]:
from crewai.tools import BaseTool


In [9]:
from pydantic import BaseModel, Field

class SentimentInput(BaseModel):
    """Tool input schema for sentiment analysis."""
    text: str = Field(..., description="Text to analyze")

class SentimentAnalysisTool(BaseTool):
    name: str = "sentiment_analysis"
    description: str = (
        "Analyzes the sentiment of a given text "
        "and returns 'positive', 'neutral', or 'negative'."
    )
    args_schema: type[BaseModel] = SentimentInput

    def _run(self, text: str) -> str:
        # (Your actual analysis logic here)
        return "positive"

In [10]:
#Sentiment analysis tool
sentiment_analysis_tool = SentimentAnalysisTool()

In [11]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, a company in the {industry} sector "
        "that recently showed interest in our solutions. Utilize all the available data sources "
        "to compile a detailed profile, focusing on key decision-makers, recent business developments, "
        "and potential needs that align with our offerings. This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Do not make assumptions and only use information you are absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, including company background, key personnel, "
        "recent milestones, and identified needs.\n"
        "Highlight potential areas where our solutions can provide value and suggest personalized "
        "engagement strategies based on the analysis."
    ),
    tools=[
        directory_read_tool,
        file_read_tool,
        search_tool
    ],
    agent=sales_rep_agent
)


In [12]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from the lead profiling report on {lead_name}, craft a personalized outreach campaign aimed at {key_decision_maker}, the {position} of {lead_name}.\n"
        "The campaign should address their recent {milestone} and how our solutions can support their goals.\n"
        "Your communication must resonate with {lead_name}'s company culture and values, demonstrating a deep understanding of their business and needs.\n"
        "Don't make assumptions and only use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts tailored to {lead_name}, specifically targeting {key_decision_maker}.\n"
        "Each draft should include a compelling narrative that connects our solutions with their recent achievements and future goals.\n"
        "Ensure the tone is engaging, professional, and aligned with {lead_name}'s corporate identity."
    ),
    tools = [
        sentiment_analysis_tool, 
        search_tool],
    agent = lead_sales_rep_agent,
)

In [13]:
crew = Crew(
    agents=[
        sales_rep_agent, 
        lead_sales_rep_agent
        ],
    tasks=[
        lead_profiling_task,
        personalized_outreach_task
        ],
        verbose = True,
        memory = True
        )

In [14]:
inputs = {
    "lead_name" : "Deeplearning.ai",
    "industry" : "AI Education",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "recently launched a new AI course"
}

result = crew.kickoff( 
    inputs=inputs
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5ceb7cb0-50dc-4b18-8af3-ecfd19d39f5f                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-08-03 07:20:08,087 - 17332 - rag_storage.py-rag_storage:132 - ERROR: Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-08-03 07:20:13,901 - 17332 - rag_storage.py-rag_storage:132 - ERROR: Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Task: Conduct an in-depth analysis of Deeplearning.ai, a company in the AI Education sector that recently      │
│  showed interest in our solutions. Utilize all the available data sources to compile a detailed profile,        │
│  focusing on key decision-makers, recent business developments, and potential needs that align with our         │
│  offerings. This task is crucial for tailoring our engagement strategy effectively.                             │
│  Do not make assumptions and only use information you are absolutely sure about.                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Thought: I need to gather information about Deeplearning.ai, including its background, key personnel, recent   │
│  developments, and potential needs that align with CrewAI's offerings. The first step to conducting this        │
│  in-depth analysis will be to search the internet for relevant and comprehensive data about Deeplearning.ai.    │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"Deeplearning.ai company profile, key decision-makers, recent business developments, pot  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  positive                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Email Draft 1: Subject: Elevating AI Education Together with CrewAI**                                        │
│                                                                                                                 │
│  Dear Andrew,                                                                                                   │
│                                                                                                                 │
│  I hope this message finds you well. Congratulations on the recent launch of your new AI course and the         │
│  exciting initiatives like the “Buildathon” at Deeplearning.ai. It's inspiring to see how you continue to lead  │
│  in the AI education landscape, making significant strides to address the growing demand for AI talent.         │
│                                                                                                                 │
│  At CrewAI, we share your commitment to advancing AI education, and we believe that our AI-driven solutions     │
│  can enhance your offerings even further. Our personalized learning technology can create tailored experiences  │
│  for your learners, helping them engage deeply with the material and come away with practical skills. Imagine   │
│  a platform where every student has access to a customized learning path, designed to meet their individual     │
│  needs and aspirations.                                                                                         │
│                                                                                                                 │
│  Additionally, our course optimization tools could assist Deeplearning.ai in analyzing user interaction data    │
│  to refine your content continuously. We're keen on collaborating with you to provide robust and flexible       │
│  solutions that align with your mission of bridging the skills gap in AI.                                       │
│                                                                                                                 │
│  I would love the opportunity to discuss how we can partner to enrich your curriculum and help your learners    │
│  excel. Would you be open to a brief call next week?                                                            │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│                                                                                                                 │
│  [Your Name]                                                                                                    │
│  Lead Sales Representative                                                                                      │
│  CrewAI                                                                                                         │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Email Draft 2: Subject: Partnering to Enhance AI Lea

2025-08-03 07:23:05,400 - 17332 - rag_storage.py-rag_storage:103 - ERROR: Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-08-03 07:23:21,858 - 17332 - rag_storage.py-rag_storage:103 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-08-03 07:23:25,282 - 17332 - rag_storage.py-rag_storage:103 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-08-03 07:23:28,349 - 17332 - rag_storage.py-rag_storage:103 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-08-03 07:23:30,710 - 17332 - rag_storage.py-rag_storage:103 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-08-03 07:23:34,391 - 17332 -

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 380129b2-28d4-49f8-bcd5-743aa379ee46                                                                     │
│  Agent: Lead Sales Representative                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5ceb7cb0-50dc-4b18-8af3-ecfd19d39f5f                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Email Draft 1: Subject: Elevating AI Education Together with CrewAI**                          │
│                                                                                                                 │
│  Dear Andrew,                                                                                                   │
│                                                                                                                 │
│  I hope this message finds you well. Congratulations on the recent launch of your new AI course and the         │
│  exciting initiatives like the “Buildathon” at Deeplearning.ai. It's inspiring to see how you continue to lead  │
│  in the AI education landscape, making significant strides to address the growing demand for AI talent.         │
│                                                                                                                 │
│  At CrewAI, we share your commitment to advancing AI education, and we believe that our AI-driven solutions     │
│  can enhance your offerings even further. Our personalized learning technology can create tailored experiences  │
│  for your learners, helping them engage deeply with the material and come away with practical skills. Imagine   │
│  a platform where every student has access to a customized learning path, designed to meet their individual     │
│  needs and aspirations.                                                                                         │
│                                                                                                                 │
│  Additionally, our course optimization tools could assist Deeplearning.ai in analyzing user interaction data    │
│  to refine your content continuously. We're keen on collaborating with you to provide robust and flexible       │
│  solutions that align with your mission of bridging the skills gap in AI.                                       │
│                                                                                                                 │
│  I would love the opportunity to discuss how we can partner to enrich your curriculum and help your learners    │
│  excel. Would you be open to a brief call next week?                                                            │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│                                                                                                                 │
│  [Your Name]                                                                                                    │
│  Lead Sales Representative                                                                                      │
│  CrewAI                                                                                                         │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                       